# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 31, 13, 20

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 4800


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4800/4800 [3:32:21<00:00,  2.65s/it]


# Check global statistics

In [1]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             15            5                  4                  3                  25                            93.229167   
                             7                  4                  5                  25                            95.025694   
                                                                   3                  25                            92.319861   
                                                                                      20                            92.592639   
                             5                  4                  3                  20                            92.778194   
                             7                  3                  5                  25                            92.466806   
                                                5                  3                  25                            92.336528   
                                                                                      20                            92.796806   
                                                                   5                  20                            94.571250   
                                                                                      25                            94.571250   
                                                4                  5                  20                            94.444444   
                                                                                      15                            97.685417   
                             5                  4                  7                  25                            96.805556   
                             7                  3                  3                  20                            85.911667   
                                                                                      25                            85.503056   
                             9                  5                  3                  20                            91.203333   
                                                3                  5                  15                            97.569444   
                             5                  3                  3                  20                            84.722083   
                                                4                  5                  25                            92.487917   
                                                                                      20                            92.487917   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             15            5                  4                  3                  25                        -1.893472   
                             7                  4                  5                  25                        -1.774167   
                                                                   3                  25                        -1.934167   
                                                                                      20                        -1.934167   
                             5                  4                  3                  20                        -1.514444   
                             7                  3                  5                  25                        -1.762778   
                                                5                  3                  25                        -1.592639   
                                                                                      20                        -1.612917   
              

# Save new config data to config file

In [6]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [3],
                            'slowd_period': [7], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

In [5]:
stat[(stat['RSI_timeperiod'] == 14) & (stat['RSI_low_bound'] == 35) & (stat['STOCH_fastk_period'] == 9) & 
          (stat['STOCH_slowk_period'] == 3) & (stat['STOCH_slowd_period'] == 7) & 
          (stat['STOCH_low_bound'] == 25)]

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,...,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
2015,STOCH_RSI,14,35,9,3,7,25,78.95,78.95,73.68,...,-0.65,-0.78,-0.76,-0.73,-0.86,19,72.367083,-0.453750,44.974583,-8.621250
3075,STOCH_RSI,14,35,9,3,7,25,80.98,63.41,64.88,...,-1.19,-1.20,-1.12,-1.01,-0.91,205,67.377917,-0.939583,-537.527083,-192.614583
2459,STOCH_RSI,14,35,9,3,7,25,87.05,70.50,66.91,...,-0.12,-0.27,-0.25,-0.23,-0.27,139,51.349583,0.021667,-2592.407917,3.011667
3075,STOCH_RSI,14,35,9,3,7,25,91.43,85.71,82.86,...,-1.25,-1.15,-0.99,-0.62,-0.30,35,74.524167,-1.129167,158.345833,-39.520833
2015,STOCH_RSI,14,35,9,3,7,25,80.56,61.11,59.72,...,-0.37,-0.41,-0.35,-0.33,-0.33,72,59.027500,-0.228750,-790.020000,-16.470000
2459,STOCH_RSI,14,35,9,3,7,25,90.10,80.20,73.27,...,-0.31,-0.32,-0.38,-0.44,-0.36,101,55.197500,0.004583,-1495.052500,0.462917
2015,STOCH_RSI,14,35,9,3,7,25,76.47,66.67,62.75,...,-0.20,-0.30,-0.22,-0.27,-0.40,51,57.352083,-0.176667,-645.043750,-9.010000
3075,STOCH_RSI,14,35,9,3,7,25,80.09,63.89,65.28,...,-1.15,-1.17,-1.10,-1.01,-0.90,216,67.418333,-0.884583,-557.640000,-191.070000
2459,STOCH_RSI,14,35,9,3,7,25,81.74,63.91,61.30,...,-0.29,-0.32,-0.40,-0.39,-0.32,230,54.837500,-0.080000,-3487.375000,-18.400000
3075,STOCH_RSI,14,35,9,3,7,25,84.21,64.91,59.65,...,0.41,0.45,0.46,0.34,0.26,57,36.987500,0.252500,-1881.712500,14.392500


In [2]:
total_stat.loc[(14, 35, 9, 3, 7, 25)]

pct_right_forecast_avg      59.748902
pct_price_diff_avg          -0.362008
forecast_rank            -1112.210871
price_rank                 -43.262538
forecasts_num             1174.000000
Name: (14, 35, 9, 3, 7, 25), dtype: float64